## Machine Learning Operations (MLOps)

In [1]:
from IPython.core.display import HTML

def load_css():
    styles = open("css/custom.css", "r").read()
    return HTML(f"<style>{styles}</style>")

load_css()

<div class="custom-slide">
    <h1>Checkpoint</h1>
    <p>
       Chegamos em um ponto do projeto em que o problema já foi explorado e a lógica do modelo em si já está bem definida. É claro que melhorias sempre irão acontecer, mas já podemos dizer que temos uma versão inicial do modelo. Daqui em diante, MLOps passa a ser cada vez mais presente e importante no processo de modelagem e deployment da solução.
    </p>
    <p>
       É preciso que fique claro que não existe uma única forma de prosseguir com o processo de modelagem após a análise exploratória do projeto. Eu resolvi cobrir dois cenários muito comuns nas empresas:
    </p>
<ol>
    <li>Registro e deployment manual do modelo</li>
    <li>Registro e Deployment do modelo por meio de plataformas que facilitam MLOps</li>
</ol>
    <p>
       Em ambas as estratégias, as seguintes tarefas são muito importantes:
    </p>
<ul>
    <li>Registrar o modelo (+ artefatos)</li>
    <li>Automatizar o pipeline de treino</li>
    <li>Automatizar o pipeline de inferência (Online e Batch serving)</li>
</ul>
    <p>
       Veja como podemos executar estas tarefas em cada uma das estratégias de registro e deployment do modelo:
    </p>
<table>
        <tr>
            <th>Tarefa</th>
            <th>(1) Registro e deployment manual do modelo</th>
            <th>(2) Registro e Deployment do modelo por meio de plataformas que facilitam MLOps</th>
        </tr>
        <tr>
            <td>Registrar o modelo (+ artefatos)</td>
            <td>Controle totalmente manual, modelo salvo em disco</td>
            <td>Controle de experimentos e registro de modelos via MLFlow</td>
        </tr>
        <tr>
            <td>Automatizar o pipeline de treino</td>
            <td>Via script</td>
            <td>Via script</td>
        </tr>
        <tr>
            <td>Automatizar o pipeline de inferência (online serving)</td>
            <td>Criação de uma API (do zero) e deployment em um ambiente escalável (Kubernetes)</td>
            <td>Utilização de frameworks de model serving (Seldon)</td>
        </tr>
        <tr>
            <td>Automatizar o pipeline de inferência (batch serving)</td>
            <td>Via script</td>
            <td>Via script</td>
        </tr>
    </table>
    <p>
       Deixaremos a implementação da estratégia (1) para o final. Por enquanto, vamos seguir em frente e analisar como seria a implementação da estratégia (2). Mas antes, gostaria de deixas algumas notas:
    </p>
    <ul>
        <li>A estratégia (1) será implementada por completo, inclusive faremos com que o modelo fique acessível na internet (como uma espécie de serviço)</li>
        <li>A estratégia (2) será implementada no localhost, simulando um ambiente corporativo. As ferramentas que vamos instalar no localhost, no ambiente corporativo, costumam estar disponíveis em algum provedor Cloud (AWS, GCP, Azure, DataBricks, etc).</li>
</ul>
<p>
       Agora sim, vamos em frente com a implementação da estratégia (2), usando uma ferramenta incrível para registro de experimenteos e modelos: <b>MLFlow</b>.
    </p>
</div>

## Importando os pacotes do projeto
Vamos centralizar nesta célula a importação de todos os pacotes que iremos utilizar neste notebook

In [2]:
from pathlib import Path
import sys 
import os

current_path = Path(os.getcwd())
parent_path = current_path.parent.absolute()
sys.path.append(str(parent_path))

import json 
import folium
import numpy as np
from sklearn.cluster import KMeans
import random
import brazilcep
from geopy.geocoders import Nominatim
import geopy.distance
from yellowbrick.cluster import KElbowVisualizer
from joblib import dump, load
import json
import matplotlib.pyplot as plt
from matplotlib import gridspec
from commons.utils import plot_points, get_delivery_coordinates, load_training_data, prepare_points, prepare_input_points, add_clusters_to_markers, get_colors, enrich_points_with_cluster_info, generate_sample_points, add_sample_points_to_markers, get_drift_params
from mlflow.models import infer_signature
from mlflow import MlflowClient
import mlflow
import json
import requests
import warnings
warnings.filterwarnings('ignore')

<div class="custom-slide">
    <div class="hands-on">
        Revisão da parte 1
    </div>
</div>

## Carregando nossos dados de treino

In [3]:
instances = load_training_data(path_str = "../data/train/")

Foram carregados 50 arquivos
O objeto instances é do tipo <class 'list'>


## Prepara os dados de treino

In [4]:
points = prepare_points(instances)
points[0]

{'lng': -47.7609277270516,
 'lat': -15.65246980577859,
 'color': 'blue',
 'cluster': None,
 'dist': None}

<div class="custom-slide">
    <div class="hands-on">
        Revisão da parte 2
    </div>
</div>

In [5]:
input_points = prepare_input_points(points)
input_points[0]

array([-15.65246981, -47.76092773])

## Carregando o modelo previamente treinado

In [6]:
# Lê o arquivo do disco
model = load('../temp/clustering_model.joblib') 
model

,n_clusters,np.int64(6)
,init,'k-means++'
,n_init,'warn'
,max_iter,300
,tol,0.0001
,verbose,0
,random_state,0
,copy_x,True
,algorithm,'lloyd'


## Cria o color map que será utilizado no mapa

In [7]:
# Seleciona as cores para usar no mapa
colors = get_colors(model.n_clusters)

# Cria um color mapa para cada cluster
cm = dict()
for cluster in np.unique(model.labels_):
    cm[cluster] = colors[cluster]

cm

{np.int32(0): '#bc0ba9',
 np.int32(1): '#323037',
 np.int32(2): '#6708bc',
 np.int32(3): '#43d381',
 np.int32(4): '#176e4a',
 np.int32(5): '#4fac4e'}

## Adiciona o centróide de cada cluster como um marcador no mapa

In [8]:
markers = add_clusters_to_markers(model)
markers

[{'lat': np.float64(-15.623188710465486),
  'lng': np.float64(-47.65238397469197),
  'cluster': 0,
  'color': 'black',
  'type': ['cluster', 'pin'],
  'tooltip': 'Cluster 0'},
 {'lat': np.float64(-15.649745950686997),
  'lng': np.float64(-47.7902749412903),
  'cluster': 1,
  'color': 'black',
  'type': ['cluster', 'pin'],
  'tooltip': 'Cluster 1'},
 {'lat': np.float64(-15.757037977498),
  'lng': np.float64(-47.771612075032984),
  'cluster': 2,
  'color': 'black',
  'type': ['cluster', 'pin'],
  'tooltip': 'Cluster 2'},
 {'lat': np.float64(-15.890221773669836),
  'lng': np.float64(-47.497399544416034),
  'cluster': 3,
  'color': 'black',
  'type': ['cluster', 'pin'],
  'tooltip': 'Cluster 3'},
 {'lat': np.float64(-15.603417375334981),
  'lng': np.float64(-47.917044479085696),
  'cluster': 4,
  'color': 'black',
  'type': ['cluster', 'pin'],
  'tooltip': 'Cluster 4'},
 {'lat': np.float64(-15.65940145501782),
  'lng': np.float64(-47.837297176334864),
  'cluster': 5,
  'color': 'black',
  

## Enriquece `points` com as informações do seu cluster


In [9]:
points = enrich_points_with_cluster_info(points, model, markers, cm)
points[0]

{'lng': -47.7609277270516,
 'lat': -15.65246980577859,
 'color': '#323037',
 'cluster': np.int32(1),
 'dist': 3.160956832617973}

<div class="custom-slide">
    <div class="hands-on">
        Revisão da parte 3
    </div>
</div>

## Implementando nosso serviço de região de entrega

In [10]:
sample_points = generate_sample_points(points, markers)
sample_points

{'covered': [{'lat': -15.65246980577859, 'lng': -47.7609277270516},
  {'lat': -15.65030404766782, 'lng': -47.75734733959318},
  {'lat': -15.646774118576014, 'lng': -47.75666348246043},
  {'lat': -15.650018577014293, 'lng': -47.757018902295506},
  {'lat': -15.65659040973423, 'lng': -47.75379878234061},
  {'lat': -15.656915561882174, 'lng': -47.75323148469429},
  {'lat': -15.65158848685651, 'lng': -47.7544944637095},
  {'lat': -15.652623182765144, 'lng': -47.761430046510455},
  {'lat': -15.659632429949554, 'lng': -47.75623444551221},
  {'lat': -15.65371700906019, 'lng': -47.75670458775543}],
 'not_covered': [{'lat': -15.668652995146136, 'lng': -47.738174492381134},
  {'lat': -15.651601758626981, 'lng': -47.71039602741898},
  {'lat': -15.645219032167478, 'lng': -47.72479997418042},
  {'lat': -15.825982398972112, 'lng': -47.577309964223566},
  {'lat': -15.969432012029449, 'lng': -47.496999028976596},
  {'lat': -15.931445419506954, 'lng': -47.39118442741347},
  {'lat': -15.880660812236018, 

Iremos salvar estes pontos de exemplo em disco, pois os utilizaremos en nossa aplicação

In [11]:
# Salva o arquivo em disco
dump(sample_points, '../temp/sample_points.joblib') 

# Lê o arquivo do disco
sample_points_new = load('../temp/sample_points.joblib') 
sample_points_new

{'covered': [{'lat': -15.65246980577859, 'lng': -47.7609277270516},
  {'lat': -15.65030404766782, 'lng': -47.75734733959318},
  {'lat': -15.646774118576014, 'lng': -47.75666348246043},
  {'lat': -15.650018577014293, 'lng': -47.757018902295506},
  {'lat': -15.65659040973423, 'lng': -47.75379878234061},
  {'lat': -15.656915561882174, 'lng': -47.75323148469429},
  {'lat': -15.65158848685651, 'lng': -47.7544944637095},
  {'lat': -15.652623182765144, 'lng': -47.761430046510455},
  {'lat': -15.659632429949554, 'lng': -47.75623444551221},
  {'lat': -15.65371700906019, 'lng': -47.75670458775543}],
 'not_covered': [{'lat': -15.668652995146136, 'lng': -47.738174492381134},
  {'lat': -15.651601758626981, 'lng': -47.71039602741898},
  {'lat': -15.645219032167478, 'lng': -47.72479997418042},
  {'lat': -15.825982398972112, 'lng': -47.577309964223566},
  {'lat': -15.969432012029449, 'lng': -47.496999028976596},
  {'lat': -15.931445419506954, 'lng': -47.39118442741347},
  {'lat': -15.880660812236018, 

Vamos então exibir estes pontos no mapa

In [12]:
# Prepara um novo marcador para incluir os novos pontos
markers_test = add_sample_points_to_markers (markers, sample_points)
#markers_test

In [ ]:
# Exibe todos os dados de treino no mapa
m = plot_points(points = points, markers = markers_test)
m

Vamos salvar este mapa como referência para o nosso modelo final

In [22]:
m.save('../temp/clustering_map.html')

<div class="custom-slide">
    <div class="hands-on">
        Revisão da parte 4
    </div>
</div>

## Analisando o erro do modelo e preparando os dados para análise do model drift (concept drift)

Primeiramente, vamos calcular as distâncias (em km) de cada ponto para o centróide de seu cluster

In [14]:
distances = dict()

for p in points:

    # Seleciona a identificação do cluster do ponto p
    cluster = p['cluster']
    
    # Adiciona o resultado à lista
    try:
        distances[cluster] += [p['dist']]
    except:
        distances[cluster] = [p['dist']]

In [15]:
drift_params = get_drift_params(distances)

In [16]:
# Salva o arquivo em disco
dump(drift_params, '../temp/drift_params.joblib') 

# Lê o arquivo do disco
drift_params_new = load('../temp/drift_params.joblib') 
drift_params_new

{np.int32(1): {'mean': np.float64(1.7),
  'stdev': np.float64(1.14),
  'perc_outliers': np.float64(0.0288),
  'perc_inner_radius': 0.9867},
 np.int32(0): {'mean': np.float64(2.62),
  'stdev': np.float64(2.7),
  'perc_outliers': np.float64(0.0318),
  'perc_inner_radius': 0.8951},
 np.int32(3): {'mean': np.float64(13.04),
  'stdev': np.float64(5.49),
  'perc_outliers': np.float64(0.048),
  'perc_inner_radius': 0.0653},
 np.int32(2): {'mean': np.float64(2.36),
  'stdev': np.float64(1.85),
  'perc_outliers': np.float64(0.0456),
  'perc_inner_radius': 0.9259},
 np.int32(5): {'mean': np.float64(2.48),
  'stdev': np.float64(1.04),
  'perc_outliers': np.float64(0.0212),
  'perc_inner_radius': 0.9901},
 np.int32(4): {'mean': np.float64(5.12),
  'stdev': np.float64(2.59),
  'perc_outliers': np.float64(0.0616),
  'perc_inner_radius': 0.5494}}

In [17]:
# Calcula a média de perc_inner_radius
model_metric = [] 
for k, v in drift_params.items():
    model_metric += [v['perc_inner_radius']] 
model_metric = np.mean(model_metric)

model_metric

np.float64(0.7354166666666667)

<div class="custom-slide">
    <div class="hands-on">
        Continuando nossa implementação
    </div>
</div>

## Registrando nosso experimento no MLFlow

Ao finalizar o desenvolvimento do seu modelo, você precisa pensar como pretende registrar o próprio modelo e seus artefatos. Isso é importante, pois o modelo será usado futuramente no pipeline de inferência. Existem duas formas principais para você pensar nesta tarefa:

1. Manualmente: neste caso, você precisaria criar uma lógica para salvar o modelo e suas dependências em algum local que possa ser usado pelo pipeline de inferência. Muitos projetos funcionam desta forma, principalmente quando não é necessário criar modelos de machine learning em massa. Lembre-se, nesta abordagem você precisar implementar tudo, inclusive versionamento dos modelos.

2. Usando uma plataforma para registro e deployment de modelos de machine learning: neste caso, plataformas como MLFlow já implementam inúmeras práticas/tarefas de registro e deployment de modelos de machine learning e você não precisa criar tudo por conta.

A escolha entre a abordagem 1 ou 2 depende muito do seu contexto de trabalho, o que inclui a escala dos modelos, orçamento, experiência, etc. É sempre bom conhecer as duas formas :) 

Neste primeiro passo, iremos registrar um experimento no MLFlow

In [18]:
os.environ['AWS_ACCESS_KEY_ID'] = 'user'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'password'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://localhost:9000'

In [23]:
mlflow.set_experiment('clustering')

artifact_path = "model-artifacts"
model_name = 'bootcamp.kmeans-clustering'

mlflow.set_tracking_uri("http://localhost:5000")

with mlflow.start_run() as run:

    run_id = run.info.run_id
    
    print("Modelo: {}".format(run_id))

    mlflow.log_params({"n_clusters": model.n_clusters})
    mlflow.log_metric("mean_perc_inner_radius", model_metric)
    mlflow.log_artifact("../temp/clustering_map.html", artifact_path = artifact_path)
    mlflow.log_artifact("../temp/drift_params.joblib", artifact_path = artifact_path)
    mlflow.log_artifact("../temp/sample_points.joblib", artifact_path = artifact_path)

    mlflow.log_input(mlflow.data.from_numpy(input_points), context="training")

    signature = infer_signature(input_points, model.predict(input_points))
    
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        signature=signature,
        input_example=input_points,
        #registered_model_name="kmeans-clustering"
    )

    print ("Completo!")

mlflow.end_run()

Modelo: 88d8efb8c65a4d619993ad2bb50c3fb6


2025/11/10 08:24:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Completo!
🏃 View run marvelous-shrike-556 at: http://localhost:5000/#/experiments/2/runs/88d8efb8c65a4d619993ad2bb50c3fb6
🧪 View experiment at: http://localhost:5000/#/experiments/2


Agora você pode iniciar o serviço do MLFlow em `http://localhost:5000`

Podemos executar novamente este notebook, alterando os parâmetros do modelo (número de clusters), e continuar registrando novos experimentos. Também poderíamos criar um loop para gerar vários experimentos de uma única vez e analisá-los na UI do MLFlow. Navamente, não existe uma regra aqui.

Fato é que, ao identificar a melhor versão do modelo que você pretende colocar em produção, você precisa registrá-la como um modelo no MLFlow (e não apenas como um experimento). Vamos assumir que a melhor versão do nosso modelo é a versão com 6 clusters e registrá-la como nossa versão final.

Ref.: https://mlflow.org/docs/latest/model-registry.html#adding-an-mlflow-model-to-the-model-registry

In [53]:
result = mlflow.register_model(
    f"runs:/{run_id}/model", "dev.{}".format(model_name)
)

result.version

Registered model 'dev.bootcamp.kmeans-clustering' already exists. Creating a new version of this model...
2024/08/13 23:22:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: dev.bootcamp.kmeans-clustering, version 17
Created version '17' of model 'dev.bootcamp.kmeans-clustering'.


'17'

Podemos também usar um client do MLFlow para acessar sua API

In [54]:
client = MlflowClient()

Agora podemos marcar a versão 1 do nosso modelo `kmeans-clustering-staging` como candidato à produção

In [55]:
# Cria um alias "candidate" para a nova versão do nosso modelo
client.set_registered_model_alias("dev.{}".format(model_name), "candidate-{}".format(run_id), result.version)

E finalmente, podemos promover este modelo para produção

In [56]:
result = client.copy_model_version(
    src_model_uri="models:/dev.{}@candidate-{}".format(model_name, run_id),
    dst_name="prod.{}".format(model_name)
)

result.version

'13'

In [57]:
# Cria um alias "champion" para a versão 1 do nosso modelo de produção
client.set_registered_model_alias("prod.{}".format(model_name), "champion", result.version)

<div class="custom-slide">
    <h1>Já está profissional!</h1>
    <p>
       Chegamos em um ponto realmente muito interessante! Toda a complexidade exigida para se manter um registro de modelo foi simplificada com MLFlow. É muito mais simples desenhar um processo de criação e registro de modelos de ML em escala, no qual é possível controlar versões dos modelos, acessar seus artefatos e métricas, assim como segmentar os ambientes de teste e produção. Fazer tudo isso de forma manual seria extremamente trabalhoso (sem contar que, muito provavelmente, a solução manual seria uma réplica do MLFlow).
    </p>
    <p>Na próxima aula, vamos resolver mais um detalhe do nosso projeto: Vamos aprender como fazer o deployment do modelo completo, não apenas do K-Means</p>
    <p>E mais uma boa notícia: iremos aprender como usar o modelo para fazer predições (tanto em modo batch, quanto em modo online)</p>
</div>